# Recommendation Algorithm for KuaiRec Video Platform

## Introduction
In this notebook, we implement a complete recommendation system using the models developed in notebook 3. Our goal is to generate personalized video recommendations for each user in the test set, leveraging the complementary strengths of our different recommendation approaches.

The recommendation algorithm combines four distinct models:
1. Collaborative Filtering (40% weight)
2. Content-Based Filtering (30% weight)
3. Sequence-Aware Model (20% weight)
4. Hybrid Model (10% weight)

Each model contributes based on its specific strengths, and the weighted ensemble approach allows us to optimize recommendations by balancing different signals from user behavior and content characteristics.

## Workflow
1. Load the trained models and test data
2. Initialize the recommendation system
3. Generate sample recommendations for a subset of users
4. Create top-10 recommendation lists for all users in the test set
5. Save the recommendations for evaluation

## 1. Setup and Data Loading

First, we import the necessary libraries and load the test data that will be used for generating recommendations.

In [11]:
# Import necessary libraries
import sys
import os
import pandas as pd
import numpy as np

# Add the src directory to the path so we can import our modules
sys.path.append(os.path.abspath("../"))
from src.recommender import KuaiRecRecommender

# Set up directories
processed_dir = "../data/processed"
models_dir = "../models"
output_dir = "../results"
os.makedirs(output_dir, exist_ok=True)

# Load test data
print("Loading test data...")
test_features = pd.read_csv(os.path.join(processed_dir, "test_features.csv"), low_memory=True)
print(f"Test features shape: {test_features.shape}")

Loading test data...
Test features shape: (934735, 21)


## 2. Initialize the Recommendation System

Now we initialize the recommender system, which loads all the previously trained models (collaborative filtering, content-based, sequence-aware, and hybrid models).

In [12]:
# Initialize the recommender
print("Initializing recommender...")
recommender = KuaiRecRecommender(models_dir=models_dir, processed_dir=processed_dir)

Initializing recommender...
Loaded interaction matrix.
Loaded collaborative filtering model.
Loaded content-based model.
Loaded sequence-aware model.
Max sequence count: 1697
Loaded hybrid model.


## 3. Generate Sample Recommendations

Let's test our recommender by generating recommendations for a small sample of users and examining the results.

In [13]:
# Generate recommendations for a few example users
print("\n--- Example Recommendations ---")
example_users = test_features['user_id'].unique()[:5]  # Use the first 5 users as examples

for user_id in example_users:
    print(f"\nRecommendations for user {user_id}:")
    
    # Get recommendations with default weights
    recs = recommender.recommend(user_id, n=5)
    
    # Display recommendations
    if recs:
        for rank, (item_id, score) in enumerate(recs):
            print(f"  {rank+1}. Video {item_id} (score: {score:.4f})")
    else:
        print("  No recommendations generated.")


--- Example Recommendations ---

Recommendations for user 14:
  1. Video 2178 (score: 1.1451)
  2. Video 1305 (score: 1.1313)
  3. Video 4123 (score: 1.0816)
  4. Video 314 (score: 0.9556)
  5. Video 4367 (score: 0.9203)

Recommendations for user 19:
  1. Video 7383 (score: 1.0405)
  2. Video 3107 (score: 0.8647)
  3. Video 2130 (score: 0.8061)
  4. Video 5811 (score: 0.8013)
  5. Video 3597 (score: 0.7969)

Recommendations for user 21:
  1. Video 4123 (score: 1.0680)
  2. Video 2130 (score: 0.8907)
  3. Video 5464 (score: 0.8684)
  4. Video 600 (score: 0.8428)
  5. Video 7191 (score: 0.7338)

Recommendations for user 23:
  1. Video 8340 (score: 1.9004)
  2. Video 1445 (score: 1.1348)
  3. Video 3742 (score: 1.0073)
  4. Video 7236 (score: 0.9737)
  5. Video 3770 (score: 0.9624)

Recommendations for user 24:
  1. Video 5291 (score: 3.9920)
  2. Video 4077 (score: 2.9249)
  3. Video 8340 (score: 1.7774)
  4. Video 497 (score: 1.1936)
  5. Video 9178 (score: 1.1907)


## 4. Generate Recommendations for All Users

Now we'll generate recommendations for all users in the test set using our weighted ensemble approach.

In [14]:
# Generate recommendations for all users in the test set
print("\n--- Generating Recommendations for All Test Users ---")
test_users = test_features['user_id'].unique()
all_recommendations = recommender.generate_recommendations_for_all_users(
    users=test_users, 
    n=10,
    weights={
        'collaborative': 0.4,
        'content': 0.3,
        'sequence': 0.2,
        'hybrid': 0.1
    }
)


--- Generating Recommendations for All Test Users ---


Generating recommendations:   0%|          | 0/1411 [00:00<?, ?it/s]

Generating recommendations: 100%|██████████| 1411/1411 [01:16<00:00, 18.38it/s]


## 5. Save Recommendations and Summarize Results

Finally, we save all recommendations to a file and provide a summary of what we've generated.

In [15]:
# Save recommendations to file
recommender.save_recommendations(
    all_recommendations, 
    os.path.join(output_dir, "recommendations.csv")
)

# Summary statistics
print("\n=== Recommendation Algorithm Summary ===")
print(f"1. Number of users with recommendations: {len(all_recommendations)}")
print(f"2. Average number of recommendations per user: {np.mean([len(recs) for recs in all_recommendations.values()]):.2f}")
print(f"3. All recommendations have been saved to {os.path.join(output_dir, 'recommendations.csv')}")
print("\nNext step: Evaluate the recommendations using appropriate metrics.")

Saved recommendations to ../results\recommendations.csv

=== Recommendation Algorithm Summary ===
1. Number of users with recommendations: 1411
2. Average number of recommendations per user: 10.00
3. All recommendations have been saved to ../results\recommendations.csv

Next step: Evaluate the recommendations using appropriate metrics.


## Analysis and Next Steps

### Results Summary
We have successfully generated personalized video recommendations for all 1,411 users in our test set, with each user receiving exactly 10 recommendations. The recommendation algorithm effectively combines the outputs from our four distinct models with the specified weights:
- 40% from Collaborative Filtering
- 30% from Content-Based Filtering
- 20% from Sequence-Aware Model
- 10% from Hybrid Model

The sample recommendations demonstrate diverse scoring patterns across different users, suggesting that the algorithm adapts to individual user preferences and viewing histories. For example, User 24's top recommendation has a significantly higher score (3.9920) compared to User 14's top recommendation (1.1451), indicating strong personalization.

### Key Achievements
1. Implemented a weighted ensemble approach that leverages all four recommendation models
2. Generated personalized top-10 recommendation lists for all test users
3. Saved recommendations in a standardized format for further evaluation
4. Processed recommendations efficiently (18.38 users per second)

### Next Steps
The next crucial phase is to evaluate these recommendations in notebook 5 using established metrics such as:
- Precision and Recall at K
- Mean Average Precision (MAP)
- Normalized Discounted Cumulative Gain (NDCG)
- Diversity and Coverage metrics

This evaluation will help us determine how well our recommendation algorithm performs and identify potential areas for improvement. We may also experiment with different model weight combinations to optimize performance further.